In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/local_Documents/TheoryAndPracticeOfCausalInference`


In [2]:
Pkg.add(["CSV", "DataFrames", "HTTP", "Statistics"])
using CSV, DataFrames, HTTP, Statistics

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


In [81]:
data04 = CSV.read(HTTP.get("https://raw.githubusercontent.com/mtakahashi123/causality/main/data04.csv").body, DataFrame; missingstring = "");

In [82]:
function attach(x::DataFrame)
    for i in propertynames(x)
        @eval $(i) = $(x).$(i)
    end
end

attach (generic function with 1 method)

In [83]:
attach(data04)

## 標準誤差

In [33]:
# 母数の集計
N₁ = 4; n₁ = 2
x₁ = [165, 166, 171, 180]
μ = mean(x₁)
hensa = @. x₁ - μ
hensa² = @. hensa^2
σ² = sum(hensa²) / N₁
σ = √σ²

5.937171043518958

In [34]:
μ # 母平均

170.5

In [35]:
σ # 母標準偏差

5.937171043518958

In [36]:
# 標本抽出と標本平均
Pkg.add("Combinatorics")
using Combinatorics

   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


In [37]:
# 標本サイズが2であるときの，とりうる標本をすべて列挙する。
xs = hcat(collect(combinations(x₁, n₁))...)

2×6 Matrix{Int64}:
 165  165  165  166  166  171
 166  171  180  171  180  180

Rのapplyと同等な，高級関数をJuliaで使うのであれば，`map()`を使えば良い。`eachcol`では`xs`の列要素のgeneratorを生成している。

In [38]:
xbars = map(mean, eachcol(xs))
# mean(xbars)
hensab = @. xbars - μ
hensa²b = @. hensab ^2
σ²b = sum(hensa²b) / 6
σb = √σ²b

3.427827300200522

In [39]:
xbars'

1×6 adjoint(::Vector{Float64}) with eltype Float64:
 165.5  168.0  172.5  168.5  173.0  175.5

In [40]:
mean(xbars) # 標本平均（不偏推定量）

170.5

In [41]:
σb # 標本標準偏差（不偏推定量ではない）

3.427827300200522

In [42]:
se₀ = σ / sqrt(n₁)

4.198213905936666

In [43]:
correct = √((N₁ - n₁) / (N₁ - 1))
se₁ = se₀ * correct

3.427827300200522

###  中心極限定理

### $\it{t}$統計量

母集団の母平均に加えて，母標準偏差も未知であるケースを考える。平均も未知であるときに，標準偏差を推定しようとすると，平均のパラメタの分だけ自由度が減る。したがって，不偏分散や標本標準偏差の分母はn-1で割られる（？？？）

標本平均をseで規格化した変数$Z = \frac{\bar{X}-\mu}{\sigma / \sqrt{n}}$は標準正規分布に従うが，このとき，分母の$\sigma$を標本標準偏差で置き換えた統計量が従う確率分布が，$t$分布である（？？？小標本の厳密な標本分布，という表現がいまいち理解できない。正規分布の無限混合分布として導出したほうがわかりやすい気もするが，とりあえずこう説明されているので，そう理解しておく）。

## 信頼区間

点推定値のまわりに誤差を加味した指標。区間推定値 (interval estimate)。

標本から推定されるパラメタのばらつきであるため，n%信頼区間は，無数にある標本（区間）のうちn%の標本において真のパラメタが含まれる区間となる。

In [44]:
Pkg.add("Distributions")
using Distributions

   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


In [50]:
cquantile(TDist(49), 0.025)

2.0095752371292397

In [53]:
quantile(TDist(49), 0.975)

2.0095752371292397

### 信頼区間による対応のある場合の2標本t検定

対応のある場合 = 同じ個体に対して2回の測定が実施されているケース。一般的には潜在的結果変数0と1は，それぞれいっぽうの処置でしか観測されないため，多重代入法などを使って補完したデータセットで推論を行う（21章）。

In [57]:
describe(data04)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,x1,81.95,58,83.5,96,0,Int64
2,y3,76.6,61,76.5,87,0,Int64
3,t1,0.3,0,0.0,1,0,Int64
4,y0,,,,84,0,String3
5,y1,,,,96,0,String3
6,y0t,73.8,52,75.0,87,0,Int64
7,y1t,83.85,61,84.5,97,0,Int64


In [58]:
function my_paired_ttest(x, y)
    n₁ = length(x) # yでも可
    diff = x - y
    m₁ = mean(diff)
    s₁ = std(diff)
    tα = cquantile(TDist(n₁ - 1), 0.025)
    return m₁ + tα * s₁ / √n₁, m₁ - tα * s₁ / √n₁
end

my_paired_ttest (generic function with 1 method)

In [59]:
my_paired_ttest(y1t, y0t)

(10.666325491505766, 9.433674508494235)

In [60]:
Pkg.add("HypothesisTests")
using HypothesisTests

   Resolving package versions...
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Project.toml`
  No Changes to `~/local_Documents/TheoryAndPracticeOfCausalInference/Manifest.toml`


対応ありの$\it{t}$検定（パッケージ版）

`my_paired_ttest()`と同じ結果が得られる。

In [61]:
OneSampleTTest(y1t, y0t)

One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0
    point estimate:          10.05
    95% confidence interval: (9.434, 10.67)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-17

Details:
    number of observations:   20
    t-statistic:              34.12951766024228
    degrees of freedom:       19
    empirical standard error: 0.2944665113655362


### 信頼区間による対応のない場合の2標本$\it{t}$検定

処置が無作為割付けであればすべての交絡因子は調整可能と仮定できる。2つの処置を受ける個体は異なるため，対応のない検定を行う。

2群の分散が等しいと仮定するかどうかで適切な検定方法が異なる。

- 等分散の検定（等しい場合）
- Welchの検定（等しくない場合）

しかし，母分散が未知である以上，この判断はできない。仮に等分散性の検定を行っても，多重検定の問題が発生する。

多重検定とは：



In [93]:
function my_Welch_test(y₀, y₁)
    y₀obs, y₁obs = @. collect(skipmissing((y₀, y₁)))
    n₀, n₁ = @. length((y₀obs, y₁obs))
    s₀, s₁ = @. std((y₀obs, y₁obs))
    num = (s₁ ^ 2 / n₁ + s₀ ^ 2 / n₀) ^ 2
    denom = ((s₁ ^ 2 / n₁) ^ 2) / (n₁ - 1) + ((s₀ ^ 2 / n₀) ^ 2) / (n₀ - 1)
    𝑑𝑓₁ = num / denom
    x̄ = mean(y₁obs) - mean(y₀obs)
    se₁ = √(s₁ ^ 2 / n₁ + s₀ ^ 2 / n₀)
    tα = cquantile(TDist(𝑑𝑓₁), 0.025)
    return x̄  + tα * se₁, x̄  - tα * se₁
end

my_Welch_test (generic function with 1 method)

In [94]:
my_Welch_test(y0, y1)

(18.469369847638223, 3.447296819028433)

In [102]:
UnequalVarianceTTest(collect(skipmissing(y1)), collect(skipmissing(y0)))

Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          10.9583
    95% confidence interval: (3.447, 18.47)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0067

Details:
    number of observations:   [12,8]
    t-statistic:              3.0692267168379623
    degrees of freedom:       17.67407317439825
    empirical standard error: 3.5703890081548075


パッケージの計算結果とも一致している。